Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [24]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [25]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [26]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    # Converts matrix to vector
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [27]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [28]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
    
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.928257
Training accuracy: 7.7%
Validation accuracy: 10.9%
Loss at step 100: 2.251825
Training accuracy: 72.3%
Validation accuracy: 71.5%
Loss at step 200: 1.816705
Training accuracy: 75.2%
Validation accuracy: 73.1%
Loss at step 300: 1.587464
Training accuracy: 76.3%
Validation accuracy: 73.8%
Loss at step 400: 1.433856
Training accuracy: 77.2%
Validation accuracy: 74.2%
Loss at step 500: 1.319759
Training accuracy: 77.9%
Validation accuracy: 74.4%
Loss at step 600: 1.229757
Training accuracy: 78.4%
Validation accuracy: 74.6%
Loss at step 700: 1.155940
Training accuracy: 78.9%
Validation accuracy: 74.5%
Loss at step 800: 1.093733
Training accuracy: 79.4%
Validation accuracy: 74.6%
Test accuracy: 82.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [29]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
  
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.936152
Minibatch accuracy: 11.7%
Validation accuracy: 12.3%
Minibatch loss at step 500: 1.475474
Minibatch accuracy: 80.5%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 1.487953
Minibatch accuracy: 78.1%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.797813
Minibatch accuracy: 78.9%
Validation accuracy: 76.9%
Minibatch loss at step 2000: 0.770262
Minibatch accuracy: 82.8%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 0.960881
Minibatch accuracy: 77.3%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.917151
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Test accuracy: 86.3%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

Logistic regression example **without** SGD: 1-hidden layer neural network with rectified linear units

In [20]:
def create_relu_graph(nb_hidden):
    graph = tf.Graph()
    with graph.as_default():
        # Input data.
        # Load the training, validation and test data into constants that are
        # attached to the graph.
        tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
        tf_train_labels = tf.constant(train_labels[:train_subset])
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
  
        # hidden layer
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, nb_hidden]))
        biases_1 = tf.Variable(tf.zeros([nb_hidden]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
        # output layer
        weights_2 = tf.Variable(tf.truncated_normal([nb_hidden, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights_2) + biases_2

        # loss function
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
        # Optimizer.
        # We are going to find the minimum of this loss using gradient descent.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
        # Predictions for the training, validation, and test data.
        # These are not part of training, but merely here so that we can report
        # accuracy figures as we train.
        def compute_prediction(dataset):
            return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = compute_prediction(tf_valid_dataset)
        test_prediction = compute_prediction(tf_test_dataset)

        return (graph, optimizer, loss, { 'train': train_prediction, 'valid': valid_prediction, 'test': test_prediction})

Compute accuracy for 10, 100, 500 and 1000 hidden nodes

In [21]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

def run_relu_computation(nb_hidden):
    print('')
    graph, optimizer, loss, preds = create_relu_graph(nb_hidden)
    with tf.Session(graph=graph) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases. 
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            _, l, predictions = session.run([optimizer, loss, preds['train']])
            if (step % 100 == 0):
                print('Loss at step %d: %f' % (step, l))
                print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
                # Calling .eval() on valid_prediction is basically like calling run(), but
                # just to get that one numpy array. Note that it recomputes all its graph
                # dependencies.
                print('Validation accuracy: %.1f%%' % accuracy(
                    preds['valid'].eval(), valid_labels))
        print('Test accuracy: %.1f%%' % accuracy(
            preds['test'].eval(), test_labels))
    
%time run_relu_computation(10)
%time run_relu_computation(100)
%time run_relu_computation(500)
%time run_relu_computation(1000)


Initialized for 10 hidden relu
Loss at step 0: 31.811394
Training accuracy: 9.5%
Validation accuracy: 15.4%
Loss at step 100: 1.547410
Training accuracy: 51.5%
Validation accuracy: 54.5%
Loss at step 200: 1.232859
Training accuracy: 65.3%
Validation accuracy: 61.5%
Loss at step 300: 1.128646
Training accuracy: 67.9%
Validation accuracy: 66.5%
Loss at step 400: 1.036702
Training accuracy: 70.5%
Validation accuracy: 69.3%
Loss at step 500: 0.989138
Training accuracy: 71.8%
Validation accuracy: 71.3%
Loss at step 600: 0.950563
Training accuracy: 72.9%
Validation accuracy: 72.5%
Loss at step 700: 0.855846
Training accuracy: 75.2%
Validation accuracy: 74.0%
Loss at step 800: 0.779935
Training accuracy: 77.5%
Validation accuracy: 75.7%
Test accuracy: 83.2%
CPU times: user 2min 3s, sys: 11.3 s, total: 2min 14s
Wall time: 42 s

Initialized for 100 hidden relu
Loss at step 0: 104.505600
Training accuracy: 13.2%
Validation accuracy: 38.8%
Loss at step 100: 2.523742
Training accuracy: 79.3%
Vali

Logistic regression example **with** SGD: 1-hidden layer neural network with rectified linear units

In [22]:
batch_size = 128

def create_sgd_relu_graph(nb_hidden):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
  
        # hidden layer
        weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, nb_hidden]))
        biases_1 = tf.Variable(tf.zeros([nb_hidden]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
        # output layer
        weights_2 = tf.Variable(tf.truncated_normal([nb_hidden, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights_2) + biases_2

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
        # Optimizer.
        # We are going to find the minimum of this loss using gradient descent.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
        # Predictions for the training, validation, and test data.
        # These are not part of training, but merely here so that we can report
        # accuracy figures as we train.
        def compute_prediction(dataset):
            return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = compute_prediction(tf_valid_dataset)
        test_prediction = compute_prediction(tf_test_dataset)
        
    return (graph, optimizer, loss, 
            { 'dataset': tf_train_dataset, 'labels': tf_train_labels },
            { 'train': train_prediction, 'valid': valid_prediction, 'test': test_prediction })

Compute accuracy for 10, 100, 500, 1000, 5000 and 10000 hidden nodes

In [23]:
num_steps = 6001

def run_sgd_relu_computation(nb_hidden):
    print('')
    graph, optimizer, loss, feed, preds = create_sgd_relu_graph(nb_hidden)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {feed['dataset'] : batch_data, feed['labels'] : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, preds['train']], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    preds['valid'].eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(preds['test'].eval(), test_labels))
    
%time run_sgd_relu_computation(10)
%time run_sgd_relu_computation(100)
%time run_sgd_relu_computation(500)
%time run_sgd_relu_computation(1000)
%time run_sgd_relu_computation(5000)
%time run_sgd_relu_computation(10000)


Initialized for 10 hidden relu
Minibatch loss at step 0: 26.358131
Minibatch accuracy: 7.8%
Validation accuracy: 14.4%
Minibatch loss at step 500: 0.991928
Minibatch accuracy: 75.8%
Validation accuracy: 64.9%
Minibatch loss at step 1000: 0.861389
Minibatch accuracy: 80.5%
Validation accuracy: 73.9%
Minibatch loss at step 1500: 0.699962
Minibatch accuracy: 82.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.684662
Minibatch accuracy: 79.7%
Validation accuracy: 75.8%
Minibatch loss at step 2500: 0.885164
Minibatch accuracy: 74.2%
Validation accuracy: 75.4%
Minibatch loss at step 3000: 0.757486
Minibatch accuracy: 73.4%
Validation accuracy: 77.9%
Minibatch loss at step 3500: 0.829638
Minibatch accuracy: 75.0%
Validation accuracy: 77.9%
Minibatch loss at step 4000: 0.738276
Minibatch accuracy: 80.5%
Validation accuracy: 77.2%
Minibatch loss at step 4500: 0.716405
Minibatch accuracy: 78.1%
Validation accuracy: 78.0%
Minibatch loss at step 5000: 0.566122
Minibatch accuracy: 82.8